<a href="https://colab.research.google.com/github/abojha/dataPrivacyLab/blob/main/cl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from scipy.stats import entropy

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DP_Lab/preprocessed_adult.csv')

In [ ]:
df.head()

,age,education,relationship,sex,race,income
0,56,HS-grad,Husband,Male,White,>50K
1,56,9th,Husband,Male,Black,<=50K
2,55,HS-grad,Unmarried,Female,Amer-Indian-Eskimo,<=50K
3,42,Doctorate,Not-in-family,Male,White,>50K
4,60,Some-college,Husband,Male,White,>50K


In [ ]:
quasi_identifiers = ["age", "education", "relationship", "sex", "race"]


df["income"] = df["income"].apply(lambda x: np.random.randint(20000, 90000) if x == "<=50K" else np.random.randint(50000, 150000))
sensitive_attribute = "income"

In [ ]:
# checking whether K-anonymous or not
def is_anony(df, k):
  count = df.groupby(quasi_identifiers).size()
  count_k = count[count < k]

  return len(count_k)

val = is_anony(df, 3)
val

2904

In [ ]:
# function for applying the k-anonymization

def a_age(age):
  if age < 30:
    return "young"
  elif age < 50:
    return "middle_aged"
  else: return "old"

def a_education(education):
  if education == "Masters" or education == "Bachelors":
    return "Higher_Studies"

  else:
    return "Lower_Studies"

def a_sex(sex):
  return "Any Sex"

def a_race(race):
  return "Any Race"

def a_relationship(relationship):
  rel = ["Husband", "Wife", "Own-child"]
  if relationship in rel:
    return "My Family"
  else : return "Non-Family"


# applying functions to the dfset
df['age'] = df['age'].apply(a_age)
df['relationship'] = df['relationship'].apply(a_relationship)
df['education'] = df['education'].apply(a_education)
df['sex'] = df['sex'].apply(a_sex)
df['race'] = df['race'].apply(a_race)

In [ ]:
# now check the k_anonymization again
val = is_anony(df, 3)
val

0

In [ ]:
df.head()

,age,education,relationship,sex,race,income
0,old,Lower_Studies,My Family,Any Sex,Any Race,60693
1,old,Lower_Studies,My Family,Any Sex,Any Race,55762
2,old,Lower_Studies,Non-Family,Any Sex,Any Race,25037
3,middle_aged,Lower_Studies,Non-Family,Any Sex,Any Race,95723
4,old,Lower_Studies,My Family,Any Sex,Any Race,102282


In [ ]:
def check_recursive_cl_diversity(group, c=1.0, l=2):
    value_counts = group["income"].value_counts().sort_values(ascending=False)
    f = value_counts.values  # Frequency array
    m = len(f)

    # If fewer than ℓ distinct sensitive values exist, fail immediately
    if m < l:
        return False, float('inf')

    # Most frequent sensitive value count
    f1 = f[0]

    # Sum of frequencies from ℓ to m
    sum_f_l_to_m = sum(f[l-1:m])

    # Avoid division by zero
    ratio = f1 / sum_f_l_to_m if sum_f_l_to_m > 0 else float('inf')

    return (f1 <= c * sum_f_l_to_m, ratio)


In [ ]:
# Group by quasi-identifiers
grouped = df.groupby(quasi_identifiers)

# Apply recursive (c, ℓ)-Diversity
diverse_groups = []
for _, group in grouped:
    r_div, ratio = check_recursive_cl_diversity(group, c=3.0, l=2)

    if r_div:
        diverse_groups.append(group)
    else:
        print(f"Group failed: Ratio={ratio:.2f}")

# Print results
if diverse_groups:
    anonymized_df = pd.concat(diverse_groups)
    print(anonymized_df)
    print("(c,l)-diversity applied Successfully")
else:
    print("No groups met Recursive (c, ℓ)-Diversity criteria.")

               age       education relationship      sex      race  income
26     middle_aged  Higher_Studies    My Family  Any Sex  Any Race  111960
29     middle_aged  Higher_Studies    My Family  Any Sex  Any Race   62569
34     middle_aged  Higher_Studies    My Family  Any Sex  Any Race   49866
70     middle_aged  Higher_Studies    My Family  Any Sex  Any Race   80650
80     middle_aged  Higher_Studies    My Family  Any Sex  Any Race   88344
...            ...             ...          ...      ...       ...     ...
15643        young   Lower_Studies   Non-Family  Any Sex  Any Race   68390
15650        young   Lower_Studies   Non-Family  Any Sex  Any Race   64441
15670        young   Lower_Studies   Non-Family  Any Sex  Any Race   62158
15672        young   Lower_Studies   Non-Family  Any Sex  Any Race   86209
15681        young   Lower_Studies   Non-Family  Any Sex  Any Race   62977

[15682 rows x 6 columns]
(c,l)-diversity applied Successfully
